<a href="https://colab.research.google.com/github/lingyixu/CS523-Deep-Learning/blob/main/GNN/GNN_Multilabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load packages

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torchmetrics

2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 10.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Dataset, Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.transforms import RandomNodeSplit
from torchmetrics.classification import BinaryHammingDistance

In [3]:
import pandas as pd
import numpy as np
import json
import glob
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [4]:
import sys
is_colab = 'google.colab' in sys.modules

if is_colab:
    from google.colab import auth
    auth.authenticate_user()

In [5]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/Shareddrives/CS523/lastFM-data/'

Mounted at /content/drive


### Load dataset: LastFM

In [6]:
df_feat = pd.read_json(drive_path + 'processed_feature.json')   # load features
arr_feat = df_feat.T.values
df_edge = pd.read_csv(drive_path + 'lastfm_asia_edges.csv')   # load edge indices
arr_edge = df_edge.T.values
df_tar = pd.read_csv(drive_path + 'lastfm_asia_target.csv', index_col=0)   # load targets
arr_tar = df_tar.values
num_features = arr_feat.shape[1]
num_classes = len(df_tar.target.unique())
classes = df_tar.target.sort_values().unique()

print('Dataset:')
print('====================')
print(f'Number of nodes: {arr_feat.shape[0]}')
print(f'Number of features: {num_features}')
print(f'Number of edges: {arr_edge.shape[1]}')
print(f'Number of distinct regions: {num_classes}')
print(f'All region classes: {classes}')

Dataset:
Number of nodes: 7624
Number of features: 7842
Number of edges: 27806
Number of distinct regions: 18
All region classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


In [7]:
graph_metadata = {'data': arr_feat, 'edge_index': arr_edge, 'label': arr_tar}

### Create GraphDataset

In [8]:
class GraphDataset(Dataset):
    
    def __init__(self, root, out_dir, metadata, transform=None, pre_transform=None, pre_filter=None):
        self.root = root
        self.out_dir = out_dir
        self.metadata = metadata
        super(GraphDataset, self).__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return root

    @property
    def processed_file_names(self):
        return glob.glob(f'{self.out_dir}/*.pt')

    def download(self):
        pass
    
    
    def process(self):

        self.node_features = torch.tensor(self.metadata['data'], dtype=torch.float)
        self.edge_index = torch.tensor(self.metadata['edge_index'], dtype=torch.int64)
        self.label = torch.tensor(self.metadata['label'], dtype=torch.int64)

        data = Data(x = self.node_features, 
                    edge_index = self.edge_index,
                    # edge_attr = None,
                    y = self.label)

        torch.save(data, f'{self.out_dir}/data.pt')

    def len(self):
        return self.node_features.shape[0]

    def get(self):
        data = torch.load(f'{self.out_dir}/data.pt')
        return data

In [9]:
root = ''
out_dir = 'graph_data'
os.mkdir(out_dir)
metadata = graph_metadata
dataset = GraphDataset(root=root, out_dir=out_dir, metadata=metadata)

Processing...
Done!


In [10]:
# inspect the graph dataset
data = dataset.get()
print()
print(data)
print('=============================================================')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')


Data(x=[7624, 7842], edge_index=[2, 27806], y=[7624, 1])
Number of nodes: 7624
Number of edges: 27806


### Train/valid/test split

In [11]:
# train_ratio = 0.6
valid_ratio = 0.2
test_ratio = 0.2
split = RandomNodeSplit(num_val = valid_ratio, num_test = test_ratio)
split_graph = split(data)

In [12]:
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Number of validation nodes: {data.val_mask.sum()}')
print(f'Number of testing nodes: {data.test_mask.sum()}')

Number of training nodes: 4574
Number of validation nodes: 1525
Number of testing nodes: 1525


### Build GNN

In [17]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels1, hidden_channels2, hidden_channels3):
        super().__init__()
        # torch.manual_seed(42)
        self.conv1 = GCNConv(num_features, hidden_channels1)
        self.conv2 = GCNConv(hidden_channels1, hidden_channels2)
        self.conv3 = GCNConv(hidden_channels2, hidden_channels3)
        self.linear = torch.nn.Linear(hidden_channels3, num_features)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        # x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.linear(x)
        x = x.sigmoid()
        return x

In [18]:
model = GCN(1024, 512, 256)
print(model)

GCN(
  (conv1): GCNConv(7842, 1024)
  (conv2): GCNConv(1024, 512)
  (conv3): GCNConv(512, 256)
  (linear): Linear(in_features=256, out_features=7842, bias=True)
)


In [19]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=5e-4)

In [20]:
def train():
      model.train()
      optimizer.zero_grad()
      out = model(data.x, data.edge_index)
      loss = criterion(out[data.train_mask], data.x[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

def test():
      test_acc = []
      model.eval()
      out = model(data.x, data.edge_index)
      metric = BinaryHammingDistance(task='multilabel', num_labels=2)   # measure average disagreement in a binary task
      test_acc = 1 - metric(out[data.test_mask], data.x[data.test_mask])
      return test_acc

for epoch in range(100):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

Epoch: 01, Loss: 0.9491
Epoch: 02, Loss: 0.9485
Epoch: 03, Loss: 0.9479
Epoch: 04, Loss: 0.9471
Epoch: 05, Loss: 0.9459
Epoch: 06, Loss: 0.9441
Epoch: 07, Loss: 0.9411
Epoch: 08, Loss: 0.9366
Epoch: 09, Loss: 0.9297
Epoch: 10, Loss: 0.9198
Epoch: 11, Loss: 0.9063
Epoch: 12, Loss: 0.8891
Epoch: 13, Loss: 0.8687
Epoch: 14, Loss: 0.8461
Epoch: 15, Loss: 0.8229
Epoch: 16, Loss: 0.8006
Epoch: 17, Loss: 0.7804
Epoch: 18, Loss: 0.7629
Epoch: 19, Loss: 0.7484
Epoch: 20, Loss: 0.7369
Epoch: 21, Loss: 0.7281
Epoch: 22, Loss: 0.7214
Epoch: 23, Loss: 0.7163
Epoch: 24, Loss: 0.7126
Epoch: 25, Loss: 0.7100
Epoch: 26, Loss: 0.7081
Epoch: 27, Loss: 0.7068
Epoch: 28, Loss: 0.7057
Epoch: 29, Loss: 0.7049
Epoch: 30, Loss: 0.7041
Epoch: 31, Loss: 0.7036
Epoch: 32, Loss: 0.7033
Epoch: 33, Loss: 0.7030
Epoch: 34, Loss: 0.7027
Epoch: 35, Loss: 0.7024
Epoch: 36, Loss: 0.7022
Epoch: 37, Loss: 0.7021
Epoch: 38, Loss: 0.7020
Epoch: 39, Loss: 0.7019
Epoch: 40, Loss: 0.7019
Epoch: 41, Loss: 0.7019
Epoch: 42, Loss:

In [21]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.9491
